In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import itertools
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LogisticRegression
import joblib

In [ ]:
galaxies = pd.read_csv('/Users/qyq/Development/Python天文培训/galaxy_standardised.csv')

In [ ]:
qsos = pd.read_csv('/Users/qyq/Development/Python天文培训/qso_standardised.csv')

In [ ]:
spectras = galaxies.append(qsos)
spectras

In [ ]:
spectras= shuffle(spectras)
spectras

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(spectras.iloc[:,0:2550],spectras['type'], test_size=0.2, random_state=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(spectras.iloc[:,0:2550],spectras['type'], test_size=0.2)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
scaler = MinMaxScaler().fit(X_train) # normalization

In [ ]:
X_train_minmax = scaler.transform(X_train)
X_test_minmax = scaler.transform(X_test)

In [ ]:
scaler = StandardScaler().fit(X_train_minmax) # standardization

In [ ]:
X_train_std = scaler.transform(X_train_minmax)
X_test_std = scaler.transform(X_test_minmax)

In [ ]:
pca = PCA(n_components=3) # Principle components analysis
pca.fit(X_train_std)
X_train_pca = pca.transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

In [ ]:
X_train_pca

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X_train_pca[:,0],X_train_pca[:,1],X_train_pca[:,2],c=y_train,alpha=0.5)
ax.set_xlabel('component 1')
ax.set_ylabel('component 2')
ax.set_zlabel('component 3')
# plt.show()

In [ ]:
# ax = plt.figure()
# plt.scatter(X_train_pca[:,0], X_train_pca[:,1],c=y_train,alpha=0.5)
# plt.xlabel('Component 1')
# plt.ylabel('Component 2')
# plt.title('Spectra after PCA(components=2)')
# plt.show()

In [ ]:
pca=PCA().fit(X_train_std)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
# plt.show()

In [ ]:
pca = PCA(n_components=500)
pca.fit(X_train_std)
X_train_pca = pca.transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
X_train_pca

In [ ]:
RandomForestClassifier?

In [ ]:
rf = RandomForestClassifier(oob_score=True)

In [ ]:
RandomForestClassifier?

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
y_predict_rf = rf.predict(X_test)

In [ ]:
y_predict_rf

In [ ]:
y_predict_proba_rf = rf.predict_proba(X_test)
y_predict_proba_rf

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test,y_predict_rf)
cnf_matrix

In [ ]:
class_names = ['galaxy','qso']
metrics.plot_confusion_matrix(rf, X_test, y_test,display_labels=class_names,
                                 cmap=plt.cm.Blues,values_format='g')

In [ ]:
print(metrics.classification_report(y_test,y_predict_rf,target_names=['galaxy','qso']))

In [ ]:
metrics.roc_auc_score(y_test, y_predict_rf)

In [ ]:
rf.oob_score_

In [ ]:
metrics.plot_roc_curve(rf, X_test, y_test)

In [ ]:
importance = rf.feature_importances_

In [ ]:
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
plt.bar([3800 + 2 * x for x in range(len(importance))], importance)
# plt.show()

In [ ]:
indices = np.argsort(importance)[::-1]
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, 3800+2*indices[f], importance[indices[f]]))

In [ ]:
cv_scores = cross_val_score(rf,X_train,y_train,cv=3)
mean = np.mean(cv_scores)

In [ ]:
mean

In [ ]:
param_test= {'n_estimators':range(10,101,10)}  
gsearch= GridSearchCV(estimator = RandomForestClassifier(),  
                       param_grid =param_test, scoring='roc_auc',cv=5)
gsearch.fit(X_train,y_train)  

In [ ]:
gsearch.best_params_

In [ ]:
gsearch.best_score_

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_pca,y_train)

In [ ]:
y_predict_proba_knn = knn.predict_proba(X_test_pca)

In [ ]:
print(f"Dimension of X_train_pca is ({len(X_train_pca)},{len(X_train_pca[0])})")

In [ ]:
print(f"Dimension of X_test_pca is ({len(X_test_pca)}, {len(X_test_pca[0])})")

In [ ]:
y_predict_proba_knn

In [ ]:
y_predict_knn = knn.predict(X_test_pca)

In [ ]:
y_predict_knn

In [ ]:
print(metrics.classification_report(y_test,y_predict_knn,target_names=['galaxy','qso']))

In [ ]:
metrics.roc_auc_score(y_test, y_predict_knn)

In [ ]:
metrics.plot_roc_curve(rf, X_test, y_test)
ax = plt.gca()
metrics.plot_roc_curve(knn, X_test_pca, y_test, ax = ax)
# plt.show()